# Initialization

In [1]:
import os
import numpy as np
import pickle
import json
import pandas as pd
from grabbit import Layout
from mne import read_epochs, grand_average, write_evokeds, read_evokeds
from mne import pick_types
from mne.viz import plot_compare_evokeds
from msit import CH_NAMES, select_subjects
import matplotlib.pyplot as plt

# load subject names to process
layout = Layout('../data', '../data/grabbit_config.json')
subjects = select_subjects(layout, 'eeg', exclude='eeg')

# load eeg configurations
with open('../config.json') as f:    
    config = json.load(f)

# load behavior
behavior = pd.read_csv('../data/derivatives/behavior/group_data.tsv', 
                       na_values='n/a', sep='\t') 
behavior = behavior[behavior.modality == 'eeg']

# Make directory structure
pipeline_root = '../data/derivatives/eeg_sensor_erps'
if not os.path.exists(pipeline_root):
    os.makedirs(pipeline_root)
for subject in subjects + ['group']:
    if not os.path.exists('%s/%s/erps' % (pipeline_root, subject)):
        os.makedirs('%s/%s/erps' % (pipeline_root, subject))

['sub-hc001', 'sub-hc002', 'sub-hc003', 'sub-hc004', 'sub-hc005', 'sub-hc006', 'sub-hc007', 'sub-hc008', 'sub-hc009', 'sub-hc010', 'sub-hc011', 'sub-hc012', 'sub-hc014', 'sub-hc015', 'sub-hc016', 'sub-hc017', 'sub-hc019', 'sub-hc020', 'sub-hc021', 'sub-hc022', 'sub-hc023', 'sub-hc024', 'sub-hc025', 'sub-hc026', 'sub-hc028', 'sub-hc029', 'sub-hc030', 'sub-hc031', 'sub-hc032', 'sub-hc033', 'sub-hc034', 'sub-hc035', 'sub-hc036', 'sub-hc037', 'sub-hc042', 'sub-hc044', 'sub-hc045', 'sub-pp001', 'sub-pp002', 'sub-pp003', 'sub-pp004', 'sub-pp005', 'sub-pp006', 'sub-pp007', 'sub-pp008', 'sub-pp009', 'sub-pp010', 'sub-pp011', 'sub-pp012', 'sub-pp013', 'sub-pp014', 'sub-pp015', 'sub-pp016']


# Make Evoked Data

In [9]:
for epo_type, epo_times in zip(config['epoch_types'], 
                               config['epoch_times']):
    
    evokeds = {'incongruent': [], 'congruent': []}
    
    for subject in subjects:
        if subject != 'sub-hc013':
            continue
        print(subject)

        sub_behavior = behavior[behavior.participant_id == subject]
        sub_behavior = sub_behavior[sub_behavior.no_response != 1].reset_index()
    
        # load, crop, and interpolate cleaned epochs
        epo_file = layout.get(subject=subject, 
                              derivative='eeg_preprocessing', 
                              extensions='%s_cleaned-epo.fif' % epo_type)[0].filename
        epochs = read_epochs(epo_file, verbose=False)
        epochs.crop(epo_times[0], epo_times[1])
        epochs.interpolate_bads(reset_bads=True)
        print(epochs)

        # drop bad eeg trials from behavior
        ar_file = layout.get(subject=subject, 
                             derivative='eeg_preprocessing', 
                             extensions='%s_ar.pkl' % epo_type)[0].filename
        ar = pickle.load(open(ar_file, 'r'))
        sub_behavior = sub_behavior.drop(ar.bad_epochs_idx).reset_index()
        
        # drop bad behavior trials from eeg and behavior
        behavior_exclude = np.where(np.sum(sub_behavior[['fast_rt', 'error', 
                                                         'post_error']], 
                                           axis=1))[0]
        sub_behavior = sub_behavior.drop(behavior_exclude)
        epochs.drop(behavior_exclude)
        
        # inject event labels
        events = sub_behavior.trial_type.astype('category').cat.codes
        if subject == 'sub-pp012':
            events = events[1:]
        epochs.event_id = {'incongruent': 1, 'congruent': 0}
        epochs.events[:, -1] = events
        print(epochs['incongruent'])
        
        # write evokeds to file
        picks = pick_types(epochs.info, eeg=True, eog=True, meg=False)
        evos = [epochs['incongruent'].average(picks), 
                epochs['congruent'].average(picks)]
        print(evos[0].ch_names)
        write_evokeds('%s/%s/%s_%s-ave.fif' % (pipeline_root,
                                               subject, subject,
                                               epo_type), evos)
        # accumulate evokeds for group average
        evokeds['incongruent'].append(evos[0])
        evokeds['congruent'].append(evos[1])
    
    # calculate group average and save
    evos = [grand_average(evokeds['incongruent']),
            grand_average(evokeds['congruent'])]
    write_evokeds('%s/group/group_%s-ave.fif' % (pipeline_root,
                                                 epo_type), evos)


print('Done!')

sub-hc013
Computing interpolation matrix from 64 sensor positions
Interpolating 6 sensors
<EpochsFIF  |  n_events : 119 (all good), tmin : -0.5 (s), tmax : 1.75 (s), baseline : None, ~176.8 MB, data loaded>
<EpochsFIF  |  n_events : 0 (all good), tmin : -0.5 (s), tmax : 1.75 (s), baseline : None, ~5.1 MB, data loaded>
evoked object is empty (based on less than 1 epoch)
[u'Fp1', u'Fpz', u'Fp2', u'AF7', u'AF3', u'AFz', u'AF4', u'AF8', u'F7', u'F5', u'F3', u'F1', u'Fz', u'F2', u'F4', u'F6', u'F8', u'FT9', u'FT7', u'FC5', u'FC3', u'FC1', u'FCz', u'FC2', u'FC4', u'FC6', u'FT8', u'FT10', u'T9', u'T7', u'C5', u'C3', u'C1', u'Cz', u'C2', u'C4', u'C6', u'T8', u'T10', u'TP9', u'TP7', u'CP5', u'CP3', u'CP1', u'CPz', u'CP2', u'CP4', u'CP6', u'TP8', u'TP10', u'P9', u'P7', u'P5', u'P3', u'P1', u'Pz', u'P2', u'P4', u'P6', u'P8', u'HEOG', u'VEOG', u'P10', u'PO7', u'PO3', u'P0z', u'PO4', u'PO8', u'O1', u'Oz', u'O2', u'Iz']
Identiying common channels ...
all channels are corresponding, nothing to do.
Id

<ipython-input-9-a9a107b884ca>:47: RuntimeWarning: evoked object is empty (based on less than 1 epoch)
  evos = [epochs['incongruent'].average(picks),


sub-hc013
Computing interpolation matrix from 64 sensor positions
Interpolating 6 sensors
<EpochsFIF  |  n_events : 105 (all good), tmin : -1.0 (s), tmax : 1.0 (s), baseline : None, ~139.8 MB, data loaded>
<EpochsFIF  |  n_events : 0 (all good), tmin : -1.0 (s), tmax : 1.0 (s), baseline : None, ~5.1 MB, data loaded>
evoked object is empty (based on less than 1 epoch)
evoked object is empty (based on less than 1 epoch)
[u'Fp1', u'Fpz', u'Fp2', u'AF7', u'AF3', u'AFz', u'AF4', u'AF8', u'F7', u'F5', u'F3', u'F1', u'Fz', u'F2', u'F4', u'F6', u'F8', u'FT9', u'FT7', u'FC5', u'FC3', u'FC1', u'FCz', u'FC2', u'FC4', u'FC6', u'FT8', u'FT10', u'T9', u'T7', u'C5', u'C3', u'C1', u'Cz', u'C2', u'C4', u'C6', u'T8', u'T10', u'TP9', u'TP7', u'CP5', u'CP3', u'CP1', u'CPz', u'CP2', u'CP4', u'CP6', u'TP8', u'TP10', u'P9', u'P7', u'P5', u'P3', u'P1', u'Pz', u'P2', u'P4', u'P6', u'P8', u'HEOG', u'VEOG', u'P10', u'PO7', u'PO3', u'P0z', u'PO4', u'PO8', u'O1', u'Oz', u'O2', u'Iz']
Identiying common channels ...

<ipython-input-9-a9a107b884ca>:47: RuntimeWarning: evoked object is empty (based on less than 1 epoch)
  evos = [epochs['incongruent'].average(picks),
<ipython-input-9-a9a107b884ca>:48: RuntimeWarning: evoked object is empty (based on less than 1 epoch)
  epochs['congruent'].average(picks)]


# Plot Congruent vs. Incongruent ERPs

In [11]:
from ipywidgets import interact

def plot_erp(subject, ch):

    fig, axs = plt.subplots(1, 2, figsize=(24, 8))

    for i, epo_type in enumerate(config['epoch_types']):

        ax = axs[i]

        evokeds = read_evokeds('%s/%s/%s_%s-ave.fif' % (pipeline_root,
                                                        subject,
                                                        subject,
                                                        epo_type),
                              verbose=False)
        pick = evokeds[0].ch_names.index(ch)
        evokeds = dict((config['conditions'][i], evokeds[i]) 
                       for i in range(len(evokeds)))
        plot_compare_evokeds(evokeds, picks=pick, axes=ax, show=False)
        ax.set_title('%s-locked' % epo_type)
        ax.set_ylim((-6, 6))
        ax.axvline(0, linestyle='--', color='k')

    plt.suptitle('%s %s ERP' % (subject, ch))
    plt.show()

interact(plot_erp, subject=subjects + ['group'], ch=CH_NAMES)

<function __main__.plot_erp>

In [7]:
plot_compare_evokeds?